# Chapter 2: Classic DP/Graphs for ML Engineers

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jmamath/interview_prep/blob/main/chapter_02_dp_graphs_ml.ipynb)

## Introduction

Have you ever wondered how Google Translate can take a sentence in one language and produce a coherent, grammatically correct translation in another? Or how a speech recognition system on your phone can accurately transcribe your spoken words into text, even in a noisy environment? These are not just feats of large-scale data processing; they are also triumphs of algorithmic ingenuity. At the heart of these technologies lie classic algorithms from computer science, adapted and scaled for the complexities of machine learning.

In this chapter, we'll pull back the curtain on some of these fundamental algorithms. We'll see how dynamic programming and graph search, concepts you might have first encountered in a standard algorithms course, are the workhorses behind many of the ML-powered features you use every day. We'll move beyond the textbook definitions and dive into practical, hands-on exercises that show you how these algorithms are applied in the real world.

By the end of this chapter, you'll not only have a deeper understanding of these classic algorithms, but you'll also have a practical toolkit for applying them to your own machine learning problems. So, let's get started!

## Learning Objectives
- Implement dynamic programming solutions for ML-related problems
- Design and implement beam search algorithms for sequence generation
- Apply graph algorithms to model training and inference problems
- Implement the Viterbi algorithm for sequence tagging
- Use diverse beam search for better generation diversity

## Chapter Progression

This chapter is organized to build your understanding progressively:

- **Foundation**: We start with an introduction to beam search, explaining the core intuition and mechanics that underlie many sequence generation tasks.
- **Problems 1-2**: You'll implement standard beam search and learn about scoring techniques like length normalization and diversity penalties.
- **Problem 3**: We shift to dynamic programming with the Viterbi algorithm, a classic method for structured prediction in sequence tagging.
- **Problems 4-5**: Finally, you'll explore advanced beam search variants that handle constraints and maximize output diversity.

---

## Beam Search: Foundation and Intuition

### What is Beam Search?

Beam search is a heuristic search algorithm that explores a graph by expanding the most promising nodes in a limited set. Unlike exhaustive search algorithms that explore every possibility, beam search keeps only the top-k most promising partial solutions at each step. This "beam" of candidates is progressively extended until complete solutions are found.

Think of beam search as a middle ground between two extremes:
- **Greedy search**: Keep only the single best option at each step (fast but often suboptimal)
- **Exhaustive search**: Keep all possibilities (optimal but computationally infeasible)

### Why Beam Search?

Consider translating "I love dogs" to French. A greedy approach might choose:
1. Pick best first word: "Je" (score: 0.9)
2. Pick best second word given "Je": "suis" (score: 0.8)
3. Result: "Je suis..." (total score: 0.72)

But the optimal translation might be:
1. Start with "J'" (score: 0.7)  
2. Add "adore" (score: 0.85)
3. Add "les chiens" (score: 0.9)
4. Result: "J'adore les chiens" (total score: 0.54) - better despite lower first word score!

**The problem**: Greedy search commits too early to locally optimal choices. By keeping multiple candidates alive (the beam), beam search can discover globally better solutions.

### Core Mechanics: A Step-by-Step Walkthrough

Let's trace beam search with a tiny vocabulary and see exactly how it works:

**Setup:**
- Vocabulary: ['hello', 'hi', 'bye']
- Beam width k=2 (keep top-2 sequences)
- Max length: 2 words
- Scoring: Longer sequences score lower (simulating log probabilities)

**Step 0 - Initialize:**
```
Beam: [(score=0.0, sequence=[])]
```

**Step 1 - First Expansion:**
Generate all possible 1-word sequences:
```
Candidates:
  (score=-1.0, ['hello'])
  (score=-0.9, ['hi'])      <- slightly better score
  (score=-1.2, ['bye'])
```
Keep top-2:
```
Beam: [(score=-0.9, ['hi']), (score=-1.0, ['hello'])]
```

**Step 2 - Second Expansion:**
Expand each sequence in the beam:
```
From ['hi']:
  (score=-1.8, ['hi', 'hello'])
  (score=-1.7, ['hi', 'hi'])
  (score=-2.1, ['hi', 'bye'])

From ['hello']:
  (score=-1.9, ['hello', 'hello'])
  (score=-1.8, ['hello', 'hi'])
  (score=-2.2, ['hello', 'bye'])
```
Keep top-2 overall:
```
Final: [(score=-1.7, ['hi', 'hi']), (score=-1.8, ['hi', 'hello'])]
```

**Key observations:**
1. We generated 6 candidates but kept only 2 (the beam width)
2. Notice 'hello' was dropped entirely - it couldn't produce competitive 2-word sequences
3. This is much faster than keeping all 3² = 9 possible sequences

### When to Use Beam Search

Beam search excels in these scenarios:
- **Machine translation**: Generating fluent translations from source text
- **Text summarization**: Producing coherent summaries of documents  
- **Speech recognition**: Converting audio to text transcriptions
- **Image captioning**: Generating natural language descriptions of images
- **Any sequence generation task** where greedy search is too myopic and exhaustive search is too expensive

### Key Parameters and Trade-offs

**Beam Width (k):**
- **k=1**: Reduces to greedy search (fast, often suboptimal)
- **k=5-10**: Common sweet spot for many applications
- **k=50+**: Approaches exhaustive search (slow, diminishing returns)

**Trade-offs:**
- **Quality vs Speed**: Larger beams find better solutions but take longer
- **Memory**: Must store k sequences and their scores at each step
- **Diversity**: Larger beams may produce similar outputs; special techniques needed for diversity

In the problems that follow, you'll implement beam search and explore techniques to balance these trade-offs.


---


## Problem 1: Simple Beam Search (Easy)

### Contextual Introduction
In machine translation or text summarization, we often need to generate a sequence of words. A simple approach, called greedy search, is to pick the most likely word at each step. However, this can lead to suboptimal results. For example, the best-scoring sentence might not start with the single best word. Beam search is a more effective alternative that keeps track of the `k` most promising sequences (the "beam") at each step, leading to better overall results.

### Key Concepts
- **Greedy Search**: Always choosing the locally optimal option at each step.
- **Beam Search**: A graph search algorithm that explores a graph by expanding the most promising nodes in a limited set.
- **Beam Width (k)**: The number of partial sequences (beams) to keep at each step.

### Problem Statement
Implement a basic beam search algorithm for sequence generation. You will be given a vocabulary and a simple scoring function. Your task is to generate the top `k` sequences of a given maximum length.

**Requirements**:
- Implement beam search with a configurable beam width.
- Support early stopping when an end-of-sequence token is reached.
- Return the top `k` sequences and their scores.

### Example: Understanding Beam Search with a Small Vocabulary

```python
# Let's see how beam search works step by step
# Vocabulary: ['a', 'b', 'c', '<END>']
# Beam width: 2 (keep top-2 sequences)
# Max length: 2

# STEP 0: Start with empty sequence
# Beam: [(score=0.0, seq=[])]

# STEP 1: Expand - try adding each word
# Candidates: (score=-1, ['a']), (score=-1, ['b']), (score=-1, ['c']), (score=-1, ['<END>'])
# After sorting by score and keeping top-2:
# Beam: [(score=-1, ['a']), (score=-1, ['b'])]

# STEP 2: Expand each sequence in beam
# From ['a']: (score=-2, ['a','a']), (score=-2, ['a','b']), (score=-2, ['a','c']), ...
# From ['b']: (score=-2, ['b','a']), (score=-2, ['b','b']), (score=-2, ['b','c']), ...
# Top-2: [(score=-2, ['a','a']), (score=-2, ['a','b'])]

# RESULT: [(['a', 'a'], -2), (['a', 'b'], -2)]

# Why this is better than greedy:
# - Greedy would pick ['a'] -> ['a','a'] only, missing other good options
# - Beam search explores multiple paths and finds better combinations
```

### Deducing the Algorithm

Now that we understand what beam search does, let's think through how to implement it. Several key design decisions need to be made:

**1. How do we maintain the beam?**

We need a data structure that can efficiently:
- Store (score, sequence) pairs
- Extract the top-k items by score
- Handle insertions as we generate new candidates

**Options:**
- Simple list with sorting: Works but O(n log n) each iteration
- Priority queue (heap): O(log n) insertions, perfect for our needs
- We'll use Python's `heapq` for efficient top-k selection

**2. What information must we track?**

Each beam entry needs:
- The **sequence** itself (list of tokens generated so far)
- The **cumulative score** (total score of this partial sequence)

Why both? We need the score for ranking but the sequence for extension and final output.

**3. When do we move sequences to "completed"?**

A sequence is complete when:
- It ends with the special `<END>` token, OR  
- It reaches maximum length

These should be separated from active sequences since they shouldn't be extended further.

**4. What are the loop termination conditions?**

The algorithm terminates when:
- We've completed `max_length` iterations, OR
- All sequences in the beam are complete (end with `<END>`)

At the end, we combine completed sequences with any remaining active sequences and return the top-k.

### Implementation Details

**Data Structures: Using `heapq` for Top-K Selection**

Python's `heapq` gives us a min-heap. Since we want the highest scores, we can either:
- Negate scores (higher scores become lower negative values)
- Use `heapq.nlargest()` for final selection

```python
import heapq
candidates = [(-1.0, ['a']), (-0.5, ['b']), (-2.0, ['c'])]
heapq.heappush(candidates, (-0.3, ['d']))  # O(log n)
best = heapq.heappop(candidates)  # Gets smallest (most negative = highest score)
```

**Score Tracking: Cumulative vs Normalized**

In this basic version, we use **cumulative scores**:
- Each token adds to the total score
- Longer sequences naturally have lower scores (if using log probabilities)
- Later, we'll add length normalization to fix this bias

**Handling the End Token**

When `<END>` is generated:
- The sequence is complete
- Move it to a separate "completed" list
- Don't expand it in future iterations

This prevents wasteful generation of sequences like `['hello', '<END>', 'world']`.

**Edge Cases**

- **beam_width=0**: Return empty list immediately
- **Empty vocabulary**: No candidates to generate, return beam as-is
- **All sequences complete early**: Stop iterating if beam is empty

**Time Complexity:** O(max_length × beam_width × |vocabulary| × log(beam_width × |vocabulary|))
- For each of max_length iterations
- Expand each of beam_width sequences  
- Generate |vocabulary| candidates
- Maintain heap of size beam_width × |vocabulary|

**Space Complexity:** O(beam_width × max_length)
- Store beam_width sequences, each up to max_length tokens

In [ ]:
from typing import List, Tuple
import heapq

class BeamSearch:
    def __init__(self, vocabulary: List[str], end_token: str = '<END>'):
        self.vocabulary = vocabulary
        self.end_token = end_token

    def score_sequence(self, sequence: List[str]) -> float:
        # In a real scenario, this would be a language model score.
        # For this exercise, we use a simple length-based score.
        return -len(sequence)

    def search(self, beam_width: int, max_length: int) -> List[Tuple[float, List[str]]]:
        # TODO: Implement the beam search algorithm.
        # Remember to handle the beam, generate candidates, and manage completed sequences.
        pass

def test_beam_search():
    vocabulary = ['a', 'b', 'c', '<END>']
    # We are providing a correct implementation here for the sake of the test
    class CorrectBeamSearch(BeamSearch):
        def search(self, beam_width: int, max_length: int) -> List[Tuple[float, List[str]]]:
            if beam_width == 0: return []
            beam = [(0.0, [])]  # (score, sequence)
            completed_sequences = []
            for _ in range(max_length):
                new_beam = []
                for score, seq in beam:
                    if not seq or seq[-1] == self.end_token:
                        completed_sequences.append((score, seq))
                        continue
                    for token in self.vocabulary:
                        new_seq = seq + [token]
                        new_score = self.score_sequence(new_seq)
                        heapq.heappush(new_beam, (new_score, new_seq))
                beam.clear()
                while new_beam and len(beam) < beam_width:
                    score, seq = heapq.heappop(new_beam)
                    beam.append((score, seq))
            all_sequences = completed_sequences + beam
            return sorted(all_sequences, key=lambda x: x[0], reverse=True)[:beam_width]

    beam_search = CorrectBeamSearch(vocabulary)

    # Test 1: Basic search
    results = beam_search.search(beam_width=2, max_length=2)
    assert len(results) == 2
    assert results[0][1] == []

    # Test 2: Zero beam width
    results = beam_search.search(beam_width=0, max_length=2)
    assert len(results) == 0

    print("🎉 All beam search tests passed!")

test_beam_search()

<details>
<summary>Click to reveal hint for Problem 1</summary>

**Hint**: Use a priority queue (like Python's `heapq`) to maintain the beam of the top `k` sequences at each step. The priority queue should store tuples of `(score, sequence)`. At each step of the generation, expand each sequence in the beam with all possible next tokens, score the new sequences, and use the priority queue to keep only the top `k`.

</details>

---

## Problem 2: Top-k Beam Search with Scores (Medium)

### Contextual Introduction
Standard beam search can sometimes produce sequences that are very similar to each other. To encourage more diversity, we can introduce techniques like length normalization and a diversity penalty. Length normalization prevents the search from favoring shorter sequences, while a diversity penalty discourages sequences that are too similar to already selected ones. These techniques are crucial for applications like creative text generation or offering multiple diverse translation options.

### Key Concepts
- **Length Normalization**: A technique to reduce the bias of beam search towards shorter sequences by dividing the score by the sequence length raised to some power.
- **Diversity Penalty**: A penalty applied to the score of a sequence based on its similarity to other sequences in the beam, encouraging more diverse outputs.

### Problem Statement
Extend the simple beam search to include length normalization and a diversity penalty. You will implement a `TopKBeamSearch` class that generates more diverse and higher-quality sequences.

**Requirements**:
- Implement length normalization in the scoring function.
- Implement a diversity penalty based on n-gram overlap.
- Combine these techniques in the search algorithm to produce diverse sequences.

### Example: Length Normalization and Diversity Penalty

```python
import math

# WITHOUT length normalization: Shorter sequences get higher scores
sequences = [
    ['hello'],           # score = -1
    ['hello', 'world']   # score = -2
]
# Result: First sequence wins! (score -1 > -2)

# WITH length normalization (dividing by length^alpha):
# alpha = 0.6 is commonly used
sequences_with_norm = [
    ['hello'],           # normalized = -1 / (1^0.6) = -1.0
    ['hello', 'world']   # normalized = -2 / (2^0.6) = -1.32
]
# Result: First still wins, but the gap is smaller

# DIVERSITY PENALTY - preventing similar sequences:
# Sequence 1: ['a', 'b', 'c']  -> bigrams: {('a','b'), ('b','c')}
# Sequence 2: ['a', 'b', 'd']  -> bigrams: {('a','b'), ('b','d')}
# Overlap: 1 bigram ('a','b')
# Diversity penalty = 1 * penalty_weight (e.g., 0.5)

# In beam search with diversity:
# - First sequence ['a', 'b', 'c'] is selected with score 1.0
# - Second sequence ['a', 'b', 'd'] gets penalty for overlapping bigram
# - New score = 0.8 - (1 * 0.5) = 0.3
# - Different sequence like ['x', 'y', 'z'] with no overlap keeps full score 0.9
# Result: More diverse outputs!
```

### Intuition: The Length Bias Problem

Standard beam search has a subtle but serious flaw: **it strongly favors shorter sequences**.

**Why does this happen?**

In most applications, scores represent log probabilities. Each additional token multiplies another probability (typically < 1) to the sequence:
- Sequence "hello": P = 0.5, log P = -0.69
- Sequence "hello world": P = 0.5 × 0.6 = 0.3, log P = -1.20

Even though the longer sequence might be better, its score is mathematically guaranteed to be lower (more negative). This creates a systematic bias against longer, potentially better outputs.

**The solution: Length Normalization**

Instead of comparing raw scores, we normalize by sequence length raised to a power α (alpha):

```
normalized_score = raw_score / (length^α)
```

**Common α values:**
- α = 0.0: No normalization (original bias)
- α = 0.6: Google's NMT default, mild normalization
- α = 1.0: Full per-token average, may over-correct

Let's see the effect with real numbers:
```
Sequence A: "hello" 
  Raw score: -0.69
  Normalized (α=0.6): -0.69 / (1^0.6) = -0.69

Sequence B: "hello world today"
  Raw score: -2.1  (looks worse!)
  Normalized (α=0.6): -2.1 / (3^0.6) = -2.1 / 2.05 = -1.02
  
After normalization, we can fairly compare: -0.69 vs -1.02
```

Without normalization, beam search often produces terse, incomplete outputs. Length normalization levels the playing field.

### Deducing the Diversity Penalty

Even with length normalization, beam search has another problem: **it produces very similar outputs**.

**Why?** The beam often gets dominated by sequences with identical prefixes:
```
Beam after step 5:
  1. "I want to go to"
  2. "I want to go home"
  3. "I want to go there"
```

All three sequences share the prefix "I want to go", so they're not giving us much diversity. For applications like offering translation alternatives or creative generation, this is problematic.

**The diversity penalty solution:**

We penalize sequences based on their similarity to sequences already in the beam. The idea:
1. Calculate n-gram overlap between candidate and existing beam sequences
2. Subtract a penalty proportional to this overlap
3. This encourages new candidates to differ from what we already have

**N-gram overlap as a similarity metric:**

Bigrams (2-grams) work well because they capture local structure:
- "the cat sat" → bigrams: {(the, cat), (cat, sat)}
- "the dog sat" → bigrams: {(the, dog), (dog, sat)}
- Overlap: 0 bigrams (0% similar despite sharing 2 words!)

```python
seq1_bigrams = {(the, cat), (cat, sat)}
seq2_bigrams = {(the, cat), (cat, runs)}
overlap = len(seq1_bigrams ∩ seq2_bigrams) = 1  # just (the, cat)
penalty = overlap × penalty_weight = 1 × 0.5 = 0.5
```

**Balancing quality and diversity:**

The `penalty_weight` parameter controls the trade-off:
- **Low weight (0.1-0.3)**: Slight diversity boost, mostly quality-focused
- **Medium weight (0.5-0.7)**: Balanced approach
- **High weight (0.8-1.0)**: Aggressive diversity, may sacrifice quality

### Implementation Details

**Computing N-grams Efficiently with `zip()`**

Python's `zip()` makes bigram extraction elegant:

```python
sequence = ['the', 'cat', 'sat']
bigrams = list(zip(sequence, sequence[1:]))
# Result: [('the', 'cat'), ('cat', 'sat')]

# For sets (fast intersection):
bigram_set = set(zip(sequence, sequence[1:]))
```

The `zip(sequence, sequence[1:])` idiom pairs each element with its successor - perfect for bigrams.

**When to Apply Penalties: Scoring vs Selection**

Two approaches:

1. **During scoring** (recommended):
   - Calculate penalty when scoring each candidate
   - Penalty = similarity to all current beam sequences
   - Pro: Natural integration into score-based selection
   - Con: Must recalculate if beam changes

2. **During selection**:
   - Generate all candidates first
   - Apply penalties only to top candidates
   - Pro: Fewer penalty calculations
   - Con: May miss diverse candidates that scored lower initially

Most implementations use approach #1 for better diversity.

**Tuning the Penalty Weight Parameter**

The penalty weight is application-specific:

- **Machine translation**: Low (0.2-0.4) - accuracy matters most
- **Creative writing**: High (0.7-1.0) - diversity is the goal  
- **Summarization**: Medium (0.4-0.6) - balance coverage and accuracy

Start with 0.5 and adjust based on output inspection. If outputs are too similar, increase; if quality drops, decrease.

**Time Complexity Addition:** O(beam_width² × max_length) for diversity calculations
- For each new candidate, compare against beam_width existing sequences
- Each comparison iterates through up to max_length bigrams

In [ ]:
from typing import List, Tuple
import math

class TopKBeamSearch(BeamSearch):
    def score_sequence(self, sequence: List[str], length_penalty: float = 0.6) -> float:
        # TODO: Implement length-normalized scoring
        pass

    def calculate_diversity_penalty(self, sequence: List[str], existing_sequences: List[List[str]], penalty_weight: float = 0.5) -> float:
        # TODO: Implement diversity penalty calculation based on n-gram overlap
        pass

    def search(self, beam_width: int, max_length: int, diversity_penalty: float = 0.5) -> List[Tuple[float, List[str]]]:
        # TODO: Implement the search method incorporating the new scoring and penalty functions
        pass

def test_top_k_beam_search():
    vocabulary = ['a', 'b', 'c', 'd', '<END>']
    # Correct implementation for testing purposes
    class CorrectTopK(TopKBeamSearch):
        def score_sequence(self, sequence: List[str], length_penalty: float = 0.6) -> float:
            raw_score = -len(sequence) * 0.5
            return raw_score / (len(sequence)**length_penalty) if sequence else 0
        def calculate_diversity_penalty(self, sequence: List[str], existing_sequences: List[List[str]], penalty_weight: float = 0.5) -> float:
            penalty = 0.0
            for existing_seq in existing_sequences:
                seq_bigrams = set(zip(sequence, sequence[1:]))
                existing_bigrams = set(zip(existing_seq, existing_seq[1:]))
                overlap = len(seq_bigrams.intersection(existing_bigrams))
                penalty += overlap * penalty_weight
            return penalty
        def search(self, beam_width: int, max_length: int, diversity_penalty: float = 0.5) -> List[Tuple[float, List[str]]]:
            beam = [(0.0, [])]
            completed = []
            for _ in range(max_length):
                new_beam = []
                for score, seq in beam:
                    if not seq or seq[-1] == self.end_token: continue
                    for token in self.vocabulary:
                        new_seq = seq + [token]
                        new_score = self.score_sequence(new_seq) - self.calculate_diversity_penalty(new_seq, [s for _, s in beam], diversity_penalty)
                        heapq.heappush(new_beam, (new_score, new_seq))
                beam.clear()
                while new_beam and len(beam) < beam_width:
                    score, seq = heapq.heappop(new_beam)
                    if seq[-1] == self.end_token: completed.append((score, seq))
                    else: beam.append((score, seq))
            return sorted(completed + beam, key=lambda x: x[0], reverse=True)[:beam_width]

    top_k_search = CorrectTopK(vocabulary)

    # Test 1: With diversity penalty, we should get more diverse results
    results_diverse = top_k_search.search(beam_width=3, max_length=3, diversity_penalty=1.0)
    results_no_diversity = top_k_search.search(beam_width=3, max_length=3, diversity_penalty=0.0)

    assert results_diverse[0][1] != results_no_diversity[0][1]

    print("🎉 All top-k beam search tests passed!")

test_top_k_beam_search()

<details>
<summary>Click to reveal hint for Problem 2</summary>

**Hint**: For length normalization, divide the sequence score by `len(sequence)**alpha`, where `alpha` is the length penalty factor. For the diversity penalty, you can calculate the n-gram overlap between a candidate sequence and the sequences already in the beam. Subtract a penalty proportional to this overlap from the candidate's score.

</details>

---

## Problem 3: Viterbi Algorithm for Sequence Tagging (Medium)

### Contextual Introduction
Part-of-speech (POS) tagging is a classic NLP task where we assign a grammatical tag (like noun, verb, or adjective) to each word in a sentence. A simple approach of picking the most likely tag for each word independently can fail because it ignores the context (e.g., "the fish" is more likely than "the and"). The Viterbi algorithm, a dynamic programming method, solves this by finding the most likely sequence of tags given the sequence of words. It's used in Hidden Markov Models (HMMs) and is fundamental to many sequence labeling tasks.

### Key Concepts
- **Hidden Markov Model (HMM)**: A statistical model with hidden states (the tags) and observable outputs (the words).
- **Transition Probability**: The probability of moving from one state to another, P(tag_i | tag_{i-1}).
- **Emission Probability**: The probability of observing a word given a state, P(word_i | tag_i).
- **Dynamic Programming**: The Viterbi algorithm uses a table to store the maximum probability of being in a certain state at a certain time, avoiding re-computation.

### Problem Statement
Implement the Viterbi algorithm for a simple POS tagger. You will be given the transition, emission, and initial probabilities of an HMM. Your task is to find the most likely sequence of POS tags for a given sentence.

**Requirements**:
- Implement the Viterbi algorithm using dynamic programming.
- Use logarithms to prevent underflow with small probabilities.
- Handle unknown words with a simple smoothing technique.
- Reconstruct the most likely path of tags.

### Example: Manual Viterbi Computation

```python
# Let's trace through a simple example: sentence = ['the', 'cat', 'sat']
# Tags: DET (determiner), NOUN (noun), VERB (verb)

# Transition probabilities P(tag_i | tag_{i-1}):
#           DET   NOUN  VERB
#  DET:    [0.1   0.8   0.1]
#  NOUN:   [0.3   0.1   0.6]
#  VERB:   [0.2   0.7   0.1]

# Emission probabilities P(word | tag):
#         'the'  'cat'  'sat'
#  DET:  [0.9    0.05   0.05]
#  NOUN: [0.05   0.9    0.05]
#  VERB: [0.05   0.05   0.9]

# Initial probabilities: [0.8 (DET), 0.1 (NOUN), 0.1 (VERB)]

# STEP 1: Process word 'the'
# For each tag, compute: initial_prob * emission_prob['the']
# DET:  0.8 * 0.9 = 0.72   <- best
# NOUN: 0.1 * 0.05 = 0.005
# VERB: 0.1 * 0.05 = 0.005
# Viterbi table: [0.72, 0.005, 0.005]

# STEP 2: Process word 'cat'
# For NOUN tag: max(
#     0.72 * 0.8 * 0.9,     <- from DET
#     0.005 * 0.1 * 0.9,    <- from NOUN
#     0.005 * 0.7 * 0.9     <- from VERB
# ) = max(0.518, 0.0005, 0.0032) = 0.518  <- from DET
# Similarly for other tags...

# STEP 3: Process word 'sat'
# Similar computation, tracking best path

# FINAL RESULT: ['DET', 'NOUN', 'VERB']
# Because this path has the highest joint probability
```

### Intuition: Why Dynamic Programming?

At first glance, finding the best tag sequence seems straightforward: just try all possibilities and pick the best. But let's examine the problem size:

**The Exponential Explosion:**

For a sentence with T words and N possible tags:
- Word 1: N choices
- Word 2: N choices  
- Word 3: N choices
- Total: N^T possible sequences

For just 10 words with 45 POS tags (typical English tagset): 45^10 ≈ 3.4 × 10^16 possibilities!

Even at 1 billion sequences/second, this would take ~1 million years to enumerate.

**The Key Insight: Optimal Substructure**

Here's the crucial observation that makes dynamic programming possible:

> The best tag sequence ending at word i with tag t depends ONLY on the best tag sequence ending at word i-1 (not the entire history).

Why? Because of the Markov property in HMMs: the probability of the current tag depends only on the previous tag, not the entire sequence before it.

**Example:**
```
Sentence: "the cat sat"
Best path to "cat" ending in NOUN:
  [DET, NOUN] with probability 0.518

To extend to "sat", we only need:
  - The probability 0.518 from the previous step
  - Transition NOUN → VERB
  - Emission probability of "sat" given VERB

We don't need to know it was specifically "the" before "cat" - 
just that we arrived at NOUN with probability 0.518.
```

This optimal substructure allows us to build the solution incrementally, reducing complexity from O(N^T) to O(T × N^2).

**Comparison to Naive Enumeration:**

```
Naive approach (3 words, 3 tags):
  DET-DET-DET, DET-DET-NOUN, DET-DET-VERB,
  DET-NOUN-DET, DET-NOUN-NOUN, ...
  Total: 27 complete sequences evaluated

Viterbi (3 words, 3 tags):
  Step 1: 3 calculations (one per tag)
  Step 2: 9 calculations (3 tags × 3 previous)  
  Step 3: 9 calculations
  Total: 21 partial sequences evaluated (and reused!)
```

The difference becomes dramatic as T grows: 27 vs 21 for T=3, but 59,049 vs 39 for T=10!

### Deducing the DP Table

Let's design the dynamic programming table step by step.

**What do we need to store?**

At each position, we need to know: "What's the highest probability of reaching this word with this tag?"

**Table Dimensions: [num_words × num_tags]**

```
        DET    NOUN   VERB
the    [0.72   0.005  0.005]  <- probabilities for word 1
cat    [0.036  0.518  0.003]  <- probabilities for word 2  
sat    [0.011  0.018  0.280]  <- probabilities for word 3
```

**What each cell represents:**

`table[word_i][tag_j]` = Maximum probability of ANY tag sequence of length i that ends with tag_j at word_i.

For example, `table[1][NOUN] = 0.518` means:
- "The best way to reach word 1 ('cat') with tag NOUN has probability 0.518"
- That path happened to be [DET, NOUN], but we don't store that yet

**Why we need backpointers:**

The table tells us the PROBABILITY of the best path, but not the path itself. We need a separate backpointer table:

```
Backpointers:     DET    NOUN   VERB
the              [-1     -1     -1]      <- no previous tag (start)
cat              [DET    DET    DET]     <- best previous tag for each
sat              [DET    NOUN   NOUN]    <- arrows pointing backward
```

`backpointer[word_i][tag_j]` = Which tag at word i-1 gave us the best path to tag_j at word_i?

**The DP Recurrence Relation:**

For each word i and tag j:

```
table[i][j] = max over all tags k of:
    table[i-1][k] × transition[k→j] × emission[word_i|j]
```

In log space (to prevent underflow):

```
table[i][j] = max over all tags k of:
    table[i-1][k] + log_transition[k→j] + log_emission[word_i|j]
```

**Forward Pass vs Backward Reconstruction:**

1. **Forward pass** (filling the table):
   - Start: word 0, use initial probabilities
   - Iterate: words 1 to T-1, apply recurrence relation
   - End: maximum value in table[T-1][:] is best probability

2. **Backward reconstruction** (following backpointers):
   - Start: argmax of table[T-1][:] gives last tag
   - Iterate: follow backpointers from end to start
   - End: complete sequence of tags

### Implementation Details

**Using Log Probabilities to Prevent Underflow**

Probabilities multiply: 0.8 × 0.7 × 0.6 × 0.5 = 0.168

But small probabilities underflow quickly:
```python
prob = 0.1 ** 50  # Result: 1e-50, close to machine epsilon!
```

**Solution:** Use logarithms to convert products to sums:
```python
log(a × b × c) = log(a) + log(b) + log(c)
log(0.8 × 0.7) = -0.22 + (-0.36) = -0.58
```

Log probabilities are negative (since probs < 1), and we want to maximize (least negative = highest prob).

```python
import numpy as np
# Add small epsilon before log to handle zeros
log_prob = np.log(prob + 1e-10)
```

**Handling Unknown Words with Smoothing**

When a word isn't in the vocabulary, we can't look up its emission probability. Simple solutions:

1. **Uniform smoothing**: Assign equal small probability to all tags
   ```python
   if word not in vocabulary:
       emission = np.full(num_tags, 1e-10)
   ```

2. **Open class assumption**: Give unknown words higher probability for NOUN, VERB (content words)

3. **Character-level features** (more advanced): Use suffix patterns like "-ing" suggests VERB

**Backtracking to Reconstruct the Path**

After filling the table, we have the best score but not the sequence:

```python
# Find best final tag
best_final_prob = np.max(table[T-1, :])
best_final_tag = np.argmax(table[T-1, :])

# Trace backward
path = [best_final_tag]
for t in range(T-1, 0, -1):
    prev_tag = backpointer[t, path[0]]
    path.insert(0, prev_tag)  # prepend
```

**Time Complexity: O(T × N^2)**

- T words in the sentence
- For each word, iterate through N tags (outer loop)
- For each tag, check N possible previous tags (inner loop)
- Result: T × N × N = O(T × N^2)

For a 20-word sentence with 45 tags: 20 × 45^2 = 40,500 operations (instant!)

Compare to naive O(N^T) = 45^20 ≈ 10^33 operations (impossible!)

**Space Complexity: O(T × N)**

- Viterbi table: T × N
- Backpointer table: T × N
- Total: 2 × T × N = O(T × N)

In [ ]:
import numpy as np
from typing import List, Tuple, Dict

class HMMTagger:
    def __init__(self, tags: List[str], words: List[str], initial_prob: np.ndarray, transition_prob: np.ndarray, emission_prob: np.ndarray):
        self.tags = tags
        self.words = words
        self.tag_to_idx = {tag: i for i, tag in enumerate(tags)}
        self.word_to_idx = {word: i for i, word in enumerate(words)}
        self.log_initial = np.log(initial_prob + 1e-10)
        self.log_transition = np.log(transition_prob + 1e-10)
        self.log_emission = np.log(emission_prob + 1e-10)

    def viterbi(self, sentence: List[str]) -> Tuple[List[str], float]:
        # TODO: Implement the Viterbi algorithm, including initialization, recursion, and backtracking.
        pass

def test_viterbi_algorithm():
    tags = ['DET', 'NOUN', 'VERB']
    words = ['the', 'cat', 'sat']
    initial_prob = np.array([0.8, 0.1, 0.1])
    transition_prob = np.array([[0.1, 0.8, 0.1], [0.3, 0.1, 0.6], [0.2, 0.7, 0.1]])
    emission_prob = np.array([[0.9, 0.05, 0.05], [0.05, 0.9, 0.05], [0.05, 0.05, 0.9]])

    class CorrectHMM(HMMTagger):
        def viterbi(self, sentence: List[str]) -> Tuple[List[str], float]:
            T = len(sentence); N = len(self.tags)
            if T == 0: return [], 0.0
            viterbi_table = np.zeros((T, N)); backpointer = np.zeros((T, N), dtype=int)
            word_idx = self.word_to_idx.get(sentence[0], -1)
            emission = self.log_emission[:, word_idx] if word_idx != -1 else np.log(np.full(N, 1e-10))
            viterbi_table[0, :] = self.log_initial + emission
            for t in range(1, T):
                word_idx = self.word_to_idx.get(sentence[t], -1)
                emission = self.log_emission[:, word_idx] if word_idx != -1 else np.log(np.full(N, 1e-10))
                for s in range(N):
                    trans_probs = viterbi_table[t-1, :] + self.log_transition[:, s]
                    viterbi_table[t, s] = np.max(trans_probs) + emission[s]
                    backpointer[t, s] = np.argmax(trans_probs)
            best_prob = np.max(viterbi_table[T-1, :]); last_state = np.argmax(viterbi_table[T-1, :])
            path = [self.tags[last_state]]
            for t in range(T - 1, 0, -1):
                last_state = backpointer[t, last_state]
                path.insert(0, self.tags[last_state])
            return path, np.exp(best_prob)

    tagger = CorrectHMM(tags, words, initial_prob, transition_prob, emission_prob)

    # Test 1: A likely sequence
    sentence1 = ['the', 'cat', 'sat']
    path, prob = tagger.viterbi(sentence1)
    assert path == ['DET', 'NOUN', 'VERB']

    # Test 2: A sequence with an unknown word
    sentence2 = ['the', 'dog', 'sat']
    path, prob = tagger.viterbi(sentence2)
    assert path == ['DET', 'NOUN', 'VERB']

    print("🎉 All Viterbi algorithm tests passed!")

test_viterbi_algorithm()

<details>
<summary>Click to reveal hint for Problem 3</summary>

**Hint**: Create a dynamic programming table of size `(num_words, num_tags)`. Each cell `(i, j)` will store the maximum probability of a tag sequence of length `i` ending with tag `j`. Also, create a `backpointer` table to store the path. Iterate through the words and for each word, calculate the probabilities for each tag based on the previous word's tag probabilities and the transition probabilities. After filling the table, backtrack from the end to find the most likely path.

</details>

---

## Problem 4: Constrained Beam Search (Medium-Hard)

### Contextual Introduction
In many real-world applications, we need to generate text that adheres to certain rules. For example, a chatbot must avoid generating toxic language, or a text summarization model might be required to include certain keywords. Constrained beam search extends the standard algorithm by pruning partial sequences that violate predefined constraints, ensuring that the final output meets the requirements.

### Key Concepts
- **Constraint Satisfaction**: The process of finding a solution that satisfies a set of constraints.
- **Hard Constraints vs. Soft Constraints**: Hard constraints must be satisfied, while soft constraints are desirable but not mandatory.
- **Pruning**: The process of eliminating partial solutions that cannot lead to a valid final solution.

### Problem Statement
Implement a constrained beam search algorithm that can handle two types of constraints: `MustContainConstraint` and `MustNotContainConstraint`. You will integrate these constraints into the beam search process to generate sequences that satisfy all given rules.

**Requirements**:
- Design an abstract `Constraint` class and specific implementations for `MustContainConstraint` and `MustNotContainConstraint`.
- Integrate constraint checking into the beam search algorithm.
- Prune the search space by discarding partial sequences that violate constraints.

### Example: How Constraints Filter Beam Search

```python
# Vocabulary: ['a', 'b', 'c', 'toxic', '<END>']
# Beam width: 2
# Constraint: MustNotContainConstraint('toxic')

# STEP 1: Generate candidates
# Candidates: (score=-1, ['a']), (score=-1, ['b']), (score=-1, ['toxic']), (score=-1, ['c'])

# STEP 2: Apply constraints
# Check each candidate:
# - ['a']: Contains 'toxic'? No -> KEEP
# - ['b']: Contains 'toxic'? No -> KEEP
# - ['toxic']: Contains 'toxic'? Yes -> PRUNE (remove)
# - ['c']: Contains 'toxic'? No -> KEEP

# STEP 3: Select top-2 after filtering
# Beam: [(score=-1, ['a']), (score=-1, ['b'])]

# STEP 4: Continue expanding only valid sequences
# From ['a']: ['a','a'], ['a','b'], ['a','c'] (not ['a','toxic']!)
# From ['b']: ['b','a'], ['b','b'], ['b','c'] (not ['b','toxic']!)

# FINAL RESULT: Only sequences without 'toxic' are generated
# This is much more efficient than generating toxic sequences and filtering later!
```

### Intuition: Early Pruning

Consider two approaches to ensuring generated text doesn't contain toxic language:

**Approach 1: Generate then Filter**
```
1. Run beam search normally
2. Generate 1000 candidate sequences
3. Filter out sequences containing banned words
4. Result: Maybe 200 sequences remain
```

**Approach 2: Prune During Search (Constrained Beam Search)**
```
1. At each step, check if adding a token violates constraints
2. Don't add invalid candidates to the beam
3. Result: Never waste time exploring invalid paths
```

**Why is early pruning so much better?**

Imagine the search space as a tree with 1000 branches at each level:
- Without pruning: Explore toxic branches for 5-10 steps, then discard (wasted computation)
- With pruning: Cut toxic branches immediately at step 1 (10,000× less work!)

**Real-world example:**

```
Generating chatbot response, max_length=20, beam_width=10, vocab_size=10000

WITHOUT constraints:
  Step 1: 10,000 candidates → keep 10
  Step 2: 100,000 candidates → keep 10
  ...
  Step 20: Generate millions of sequences, then filter
  
WITH constraints (pruning toxic words):
  Step 1: 10,000 candidates → filter 50 toxic → keep 10 safe ones
  Step 2: Only expand 10 safe sequences (not toxic branches!)
  ...
  Step 20: Never explored toxic paths, saved ~90% computation
```

**How constraints reduce the search space:**

Think of constraints as pruning shears in a search tree:
- Early cuts prevent entire subtrees from being explored
- The earlier you cut, the more you save (exponential savings)
- Each constraint violation detected at depth d saves vocab_size^(max_depth-d) candidate evaluations

### Deducing the Design

How should we design a system that supports multiple types of constraints?

**Design Decision 1: Abstract Constraint Interface**

We want to support many constraint types:
- MustContain: "Response must include the user's name"
- MustNotContain: "No profanity allowed"
- MaxLength: "Keep it under 50 characters"
- RequireFormat: "Must be valid JSON"

**Solution:** Abstract base class with a single interface:

```python
class Constraint(ABC):
    @abstractmethod
    def check(self, sequence: List[str]) -> bool:
        """Returns True if sequence satisfies constraint"""
        pass
```

This allows us to add new constraints without modifying the beam search code!

**Design Decision 2: When to Check Constraints?**

Three options:

1. **Check every candidate** (recommended):
   - During beam expansion, test each new sequence
   - Pro: Maximum pruning, catches violations immediately
   - Con: Most constraint checks
   
2. **Check every step** (too early):
   - Before expanding, check current beam
   - Pro: Fewer checks
   - Con: Misses violations introduced by next token

3. **Check at the end** (too late):
   - After search completes, filter results
   - Pro: Fewest checks
   - Con: No pruning benefit at all!

Option 1 is best: we check each candidate right after generation, before adding to the beam.

**Design Decision 3: Combining Multiple Constraints**

When we have multiple constraints, how do we combine them?

**AND Logic (all must be satisfied):**
```python
valid = all(constraint.check(seq) for constraint in constraints)
```

This is almost always what we want. The sequence must satisfy ALL constraints.

**OR Logic (at least one must be satisfied):**
```python
valid = any(constraint.check(seq) for constraint in constraints)
```

Rarely used, but could be implemented with a `CompositeConstraint` class if needed.

### Implementation Details

**Using Abstract Base Class for `Constraint`**

Python's `abc` module provides clean interface definition:

```python
from abc import ABC, abstractmethod

class Constraint(ABC):
    @abstractmethod
    def check(self, sequence: List[str]) -> bool:
        pass

# Concrete implementations:
class MustContainConstraint(Constraint):
    def __init__(self, required_token: str):
        self.required_token = required_token
    
    def check(self, sequence: List[str]) -> bool:
        return self.required_token in sequence

class MustNotContainConstraint(Constraint):
    def __init__(self, forbidden_token: str):
        self.forbidden_token = forbidden_token
    
    def check(self, sequence: List[str]) -> bool:
        return self.forbidden_token not in sequence
```

The key insight: both implement the same `check()` interface but with different logic.

**Constraint Checking in the Expansion Loop**

Integration point in beam search:

```python
for score, seq in beam:
    for token in vocabulary:
        new_seq = seq + [token]
        
        # CHECK CONSTRAINTS HERE (before scoring/adding to beam)
        if all(constraint.check(new_seq) for constraint in self.constraints):
            new_score = self.score_sequence(new_seq)
            heapq.heappush(new_beam, (new_score, new_seq))
        # Else: skip this candidate entirely (pruned!)
```

Notice: we never score or consider sequences that violate constraints.

**Performance Implications of Constraint Complexity**

Different constraints have different computational costs:

1. **MustNotContain**: O(sequence_length)
   - Simple membership check: `token in sequence`
   - Very fast, negligible overhead

2. **MustContain**: O(sequence_length)  
   - Same as MustNotContain
   - But may allow fewer valid sequences (more pruning)

3. **Pattern matching** (regex): O(sequence_length × pattern_complexity)
   - More expensive, but still usually acceptable
   - Example: "Must match email format"

4. **External validation** (API call): O(network_latency)
   - Very expensive! Avoid in inner loop
   - Example: "Check if URL is valid" - better to validate after search

**Best practice:** Keep constraint checks lightweight. If validation is expensive, do it once after search rather than for every candidate.

**Constraint Violation Tracking (Optional Enhancement)**

For debugging, you might track why sequences were rejected:

```python
class ConstraintViolationTracker:
    def __init__(self):
        self.violations = defaultdict(int)
    
    def check_and_track(self, seq, constraints):
        for constraint in constraints:
            if not constraint.check(seq):
                self.violations[constraint.__class__.__name__] += 1
                return False
        return True

# Analysis:
# MustNotContainConstraint: 1,245 sequences rejected
# MustContainConstraint: 89 sequences rejected
```

This helps identify which constraints are most restrictive.

In [ ]:
from typing import List, Tuple, Set
from abc import ABC, abstractmethod

class Constraint(ABC):
    @abstractmethod
    def check(self, sequence: List[str]) -> bool:
        pass

class MustContainConstraint(Constraint):
    def __init__(self, required_token: str):
        self.required_token = required_token

    def check(self, sequence: List[str]) -> bool:
        # TODO: Implement the check for must-contain constraint
        pass

class MustNotContainConstraint(Constraint):
    def __init__(self, forbidden_token: str):
        self.forbidden_token = forbidden_token

    def check(self, sequence: List[str]) -> bool:
        # TODO: Implement the check for must-not-contain constraint
        pass

class ConstrainedBeamSearch(BeamSearch):
    def __init__(self, vocabulary: List[str], end_token: str = '<END>'):
        super().__init__(vocabulary, end_token)
        self.constraints = []

    def add_constraint(self, constraint: Constraint):
        self.constraints.append(constraint)

    def search(self, beam_width: int, max_length: int) -> List[Tuple[float, List[str]]]:
        # TODO: Implement the constrained beam search
        pass

def test_constrained_beam_search():
    vocabulary = ['a', 'b', 'c', 'd', '<END>']
    class CorrectConstrained(ConstrainedBeamSearch):
        def search(self, beam_width: int, max_length: int) -> List[Tuple[float, List[str]]]:
            beam = [(0.0, [])]
            completed = []
            for _ in range(max_length):
                new_beam = []
                for score, seq in beam:
                    if not seq or seq[-1] == self.end_token: continue
                    for token in self.vocabulary:
                        new_seq = seq + [token]
                        if all(c.check(new_seq) for c in self.constraints):
                            new_score = self.score_sequence(new_seq)
                            heapq.heappush(new_beam, (new_score, new_seq))
                beam.clear()
                while new_beam and len(beam) < beam_width:
                    score, seq = heapq.heappop(new_beam)
                    if seq[-1] == self.end_token: completed.append((score, seq))
                    else: beam.append((score, seq))
            return sorted(completed + beam, key=lambda x: x[0], reverse=True)[:beam_width]
    
    # Test 1: Must contain 'c'
    searcher_must_contain = CorrectConstrained(vocabulary)
    searcher_must_contain.add_constraint(MustContainConstraint('c'))
    results = searcher_must_contain.search(beam_width=2, max_length=3)
    for _, seq in results:
        assert 'c' in seq

    # Test 2: Must not contain 'b'
    searcher_must_not_contain = CorrectConstrained(vocabulary)
    searcher_must_not_contain.add_constraint(MustNotContainConstraint('b'))
    results = searcher_must_not_contain.search(beam_width=2, max_length=3)
    for _, seq in results:
        assert 'b' not in seq

    print("🎉 All constrained beam search tests passed!")

test_constrained_beam_search()

<details>
<summary>Click to reveal hint for Problem 4</summary>

**Hint**: Create an abstract base class `Constraint` with a `check` method. Then, implement concrete constraint classes like `MustContainConstraint` and `MustNotContainConstraint`. In your beam search loop, after generating a new candidate sequence, iterate through your list of constraints and only add the sequence to the new beam if it satisfies all of them.

</details>

---

## Problem 5: Diverse Beam Search with Groups (Hard)

### Contextual Introduction
While standard beam search is good at finding high-quality sequences, it often produces a set of very similar results. For creative applications like story generation or offering multiple translation choices, we need diversity. Diverse beam search addresses this by partitioning the beam into groups and encouraging each group to explore a different part of the search space. This ensures that the final set of sequences is both high-quality and diverse.

### Key Concepts
- **Sequence Similarity**: A metric to quantify how similar two sequences are (e.g., n-gram overlap, Jaccard similarity).
- **Clustering/Grouping**: The process of partitioning a set of items into groups based on similarity.
- **Quality-Diversity Trade-off**: The balance between generating high-scoring (quality) sequences and generating a wide variety of (diverse) sequences.

### Problem Statement
Implement a diverse beam search algorithm that groups similar sequences and selects the best from each group. This will involve calculating sequence similarity, grouping sequences, and modifying the beam search to maintain diversity across groups.

**Requirements**:
- Implement a function to calculate sequence similarity (e.g., Jaccard similarity of bigrams).
- In each step of the beam search, group the candidate sequences.
- Select the best sequence from each group to form the new beam, ensuring diversity.

### Example: Sequence Similarity and Grouping

```python
# Calculating Jaccard similarity between sequences using bigrams

# Sequence 1: ['hello', 'world', 'end']
# Bigrams: {('hello', 'world'), ('world', 'end')}

# Sequence 2: ['hello', 'world', 'now']
# Bigrams: {('hello', 'world'), ('world', 'now')}

# Intersection: {('hello', 'world')}  (1 common bigram)
# Union: {('hello', 'world'), ('world', 'end'), ('world', 'now')}  (3 total unique)
# Jaccard similarity = 1 / 3 ≈ 0.33  (quite similar)

# Sequence 3: ['goodbye', 'friend', 'soon']
# Bigrams: {('goodbye', 'friend'), ('friend', 'soon')}

# Intersection with Seq1: {} (0 common bigrams)
# Union with Seq1: {('hello', 'world'), ('world', 'end'), ('goodbye', 'friend'), ('friend', 'soon')}
# Jaccard similarity = 0 / 4 = 0.0  (very different!)

# GROUPING IN DIVERSE BEAM SEARCH:
# Beam width: 4, Groups: 2
# Candidates (sorted by score):
# 1. (score=0.9, ['hello', 'world', 'end'])
# 2. (score=0.88, ['hello', 'world', 'now'])      <- Similar to #1
# 3. (score=0.85, ['goodbye', 'friend', 'soon'])  <- Different from #1
# 4. (score=0.82, ['hi', 'there', 'friend'])      <- Different from #1 and #3

# GROUP 1 (from sequence 1): [#1, #2]  <- Similar sequences
# GROUP 2 (from sequence 3): [#3, #4]  <- Different sequences

# SELECT BEST FROM EACH GROUP:
# From GROUP 1: #1 (best score)
# From GROUP 2: #3 (best score in this group)

# RESULT: [['hello', 'world', 'end'], ['goodbye', 'friend', 'soon']]
# More diverse than ['hello', 'world', 'end'], ['hello', 'world', 'now']
```

### Intuition: Quality-Diversity Trade-off

Standard beam search has a fundamental limitation: **it converges to similar high-probability sequences**.

**Why does this happen?**

Beam search is designed to find the highest-scoring sequences. Often, the top-10 sequences are variations of the same underlying path:

```
Translation task - beam_width=5:
1. "I would like to go home" (score: 0.95)
2. "I would like to go back home" (score: 0.94)
3. "I would like to return home" (score: 0.93)
4. "I'd like to go home" (score: 0.92)
5. "I want to go home" (score: 0.91)
```

All five are essentially the same translation! Not useful if we want to show the user diverse alternatives.

**The Quality-Diversity Trade-off:**

There's a tension between two goals:
- **Quality**: Generate sequences with high probability scores
- **Diversity**: Generate sequences that differ meaningfully from each other

Naive approaches fail at both ends of the spectrum:
- Pure quality focus → all outputs look the same (current beam search)
- Pure diversity focus → random sampling of low-quality sequences (useless outputs)

**What we want:** A sweet spot that maintains reasonably high quality while ensuring outputs are meaningfully different.

**The grouping concept:**

Instead of one beam competing for all positions, divide the beam into groups:
- Group 1 explores one region of the search space (e.g., formal language)
- Group 2 explores a different region (e.g., casual language)
- Group 3 explores another region (e.g., concise expressions)

Each group maintains high quality within its region, but groups diverge from each other.

**Real-world analogy:**

Think of diverse beam search like a restaurant recommendation system:
- Standard beam search: Top 5 results are all Italian restaurants (highest rated)
- Diverse beam search: Top 5 includes Italian, Thai, Mexican, Japanese, French (high rated AND diverse)

### Deducing Jaccard Similarity

To encourage diversity, we need a way to measure how similar two sequences are. Why not just count shared words?

**Problem with word overlap:**

```
Sequence A: "the cat sat on the mat"
Sequence B: "the dog sat on the mat"
Word overlap: 5/6 words match (83% similar?)

But they're fundamentally different! Different subjects doing different things.
```

**Why bigrams capture similarity better:**

Bigrams preserve word order and local context:

```
Sequence A: "the cat sat"
  Words: {the, cat, sat}
  Bigrams: {(the,cat), (cat,sat)}

Sequence B: "the dog sat"  
  Words: {the, dog, sat}  (66% overlap with A)
  Bigrams: {(the,dog), (dog,sat)}  (0% overlap with A!)
```

The bigram overlap correctly identifies these as different sequences despite sharing 2/3 words.

**Jaccard Similarity Formula:**

For two sets A and B:
```
Jaccard(A, B) = |A ∩ B| / |A ∪ B|
              = (size of intersection) / (size of union)
```

Values range from 0 (completely different) to 1 (identical).

**Why Jaccard over other similarity metrics?**

1. **Cosine similarity**: Requires vector representations, more complex
2. **Edit distance**: Sensitive to insertions/deletions, not length-normalized
3. **Jaccard**: Simple, intuitive, naturally normalized to [0,1]

**Practical example:**

```python
seq1 = ['I', 'love', 'machine', 'learning']
seq2 = ['I', 'love', 'deep', 'learning']

bigrams1 = {('I','love'), ('love','machine'), ('machine','learning')}
bigrams2 = {('I','love'), ('love','deep'), ('deep','learning')}

intersection = {('I','love')}  # 1 bigram
union = {('I','love'), ('love','machine'), ('machine','learning'), 
         ('love','deep'), ('deep','learning')}  # 5 bigrams

Jaccard = 1/5 = 0.2  (20% similar)
```

**Threshold for "similar" sequences:**

- Jaccard > 0.5: Very similar, probably in the same group
- Jaccard 0.2-0.5: Moderate similarity, borderline
- Jaccard < 0.2: Different enough for separate groups

### Implementation Details

**Maintaining Multiple Group Beams vs Single Beam with Penalties**

Two architectural approaches:

**Approach 1: Separate beams per group**
```python
beams = [
    [(score1, seq1), (score2, seq2)],  # Group 1 beam
    [(score3, seq3), (score4, seq4)],  # Group 2 beam
    [(score5, seq5), (score6, seq6)]   # Group 3 beam
]
```
- Pro: Clean separation, easy to understand
- Con: More complex bookkeeping, groups may become unbalanced

**Approach 2: Single beam with group penalties**
```python
beam = [
    (score1 - group1_penalty, seq1, group=1),
    (score2 - group2_penalty, seq2, group=2),
    ...
]
```
- Pro: Simpler data structure, natural competition
- Con: Penalty tuning is critical

Most implementations use Approach 2 for simplicity. The group penalty creates implicit separation:

```python
# Group i gets penalty i * diversity_strength
new_score = base_score - (group_index * diversity_strength)
```

This makes later groups start with a handicap, forcing them to explore different (lower-scoring but diverse) regions.

**Selecting Representatives from Each Group**

After generating candidates, how do we assign them to groups?

**Simple hash-based assignment:**
```python
group_index = hash(' '.join(sequence[:k])) % num_groups
```
This assigns sequences based on their first k tokens, naturally clustering similar prefixes.

**Similarity-based assignment (more sophisticated):**
```python
for candidate in candidates:
    # Find most similar existing group
    similarities = [jaccard(candidate, group_rep) for group_rep in group_representatives]
    best_group = argmax(similarities)
    
    if similarities[best_group] > threshold:
        add_to_group(best_group, candidate)
    else:
        create_new_group(candidate)  # Start new group
```

**Tuning the `diversity_strength` Parameter**

This parameter controls how aggressively we push for diversity:

**diversity_strength = 0.0:**
- No penalty differences between groups
- Reduces to standard beam search
- Result: All groups converge to similar sequences

**diversity_strength = 0.5 (moderate):**
- Group 0: no penalty
- Group 1: -0.5 score penalty  
- Group 2: -1.0 score penalty
- Result: Balanced quality and diversity

**diversity_strength = 2.0 (aggressive):**
- Group 0: no penalty
- Group 1: -2.0 penalty (significant!)
- Group 2: -4.0 penalty (drastic!)
- Result: High diversity, but lower-quality sequences

**Rule of thumb:** Start with diversity_strength = 0.5. If outputs are too similar, increase to 0.7-1.0. If quality suffers, decrease to 0.2-0.4.

**Computational Complexity:**

Diverse beam search with G groups and beam width K:
- Each group maintains K/G sequences
- Expansion: same as standard beam search O(K × |V| × max_length)
- Similarity calculation: O(G × max_length) per candidate if using Jaccard
- **Total:** Roughly same complexity as standard beam search + O(G × max_length) overhead

The diversity overhead is typically small compared to the base search cost.

In [ ]:
from typing import List, Tuple
from collections import defaultdict

class DiverseBeamSearch(BeamSearch):
    def calculate_similarity(self, seq1: List[str], seq2: List[str]) -> float:
        # TODO: Implement Jaccard similarity for bigrams
        pass

    def search(self, beam_width: int, max_length: int, num_groups: int, diversity_strength: float = 0.5) -> List[Tuple[float, List[str]]]:
        # TODO: Implement diverse beam search using groups
        pass

def test_diverse_beam_search():
    vocabulary = ['a', 'b', 'c', 'd', 'e', '<END>']
    class CorrectDiverse(DiverseBeamSearch):
        def calculate_similarity(self, seq1: List[str], seq2: List[str]) -> float:
            set1 = set(zip(seq1, seq1[1:])); set2 = set(zip(seq2, seq2[1:]))
            intersection = len(set1.intersection(set2)); union = len(set1.union(set2))
            return intersection / union if union > 0 else 0
        def search(self, beam_width: int, max_length: int, num_groups: int, diversity_strength: float = 0.5) -> List[Tuple[float, List[str]]]:
            if num_groups > beam_width: num_groups = beam_width
            beams = [[(0.0, [])] for _ in range(num_groups)]
            completed = []
            for _ in range(max_length):
                all_candidates = []
                for i in range(num_groups):
                    for score, seq in beams[i]:
                        if not seq or seq[-1] == self.end_token: continue
                        for token in self.vocabulary:
                            new_seq = seq + [token]
                            new_score = self.score_sequence(new_seq) - (i * diversity_strength)
                            all_candidates.append((new_score, new_seq))
                beams = [[] for _ in range(num_groups)]
                sorted_candidates = sorted(all_candidates, key=lambda x: x[0], reverse=True)
                for score, seq in sorted_candidates:
                    group_idx = hash(' '.join(seq[:1])) % num_groups
                    if len(beams[group_idx]) < beam_width / num_groups:
                        if seq[-1] == self.end_token: completed.append((score, seq))
                        else: beams[group_idx].append((score, seq))
            flat_beam = [item for sublist in beams for item in sublist]
            return sorted(completed + flat_beam, key=lambda x: x[0], reverse=True)[:beam_width]

    diverse_searcher = CorrectDiverse(vocabulary)

    # Test that with diversity, we get different results
    results_diverse = diverse_searcher.search(beam_width=4, max_length=3, num_groups=2, diversity_strength=0.8)
    results_regular = diverse_searcher.search(beam_width=4, max_length=3, num_groups=1, diversity_strength=0.0)

    assert results_diverse[0][1] != results_regular[0][1]

    print("🎉 All diverse beam search tests passed!")

test_diverse_beam_search()

<details>
<summary>Click to reveal hint for Problem 5</summary>

**Hint**: A simple approach to grouping is to use a penalty. In each step of the beam search, when you are scoring new candidate sequences, add a penalty to the score that is proportional to the sequence's similarity to the other sequences in its group. This will encourage the groups to diverge. For example, you can use `group_index * diversity_penalty` as a simple penalty.

</details>